In [1]:
import cv2
import pandas as pd
import os

# Folder containing videos and CSV files
folder_path = 'raw_data'

# Get a list of all files in the folder
files = os.listdir(folder_path)

# Create a directory to store frames and labels
output_dir = 'output_frames'
os.makedirs(output_dir, exist_ok=True)

# Iterate over the files in the folder
for file in files:
    # Check if the file is a video file
    if file.endswith('.mp4'):
        # Get the video file path
        video_file = os.path.join(folder_path, file)

        # Get the corresponding CSV file path
        csv_file = os.path.join(folder_path, file[:-4] + '.csv')

        # Read the video file
        vidcap = cv2.VideoCapture(video_file)

        # Get the frame rate of the original video
        frame_rate = vidcap.get(cv2.CAP_PROP_FPS)

        # Read the CSV file containing frame information
        df = pd.read_csv(csv_file)

        success, image = vidcap.read()
        count = 0

        while success:
            # Save the frame image
            frame_filename = os.path.join(output_dir, f'{file[:-4]}_frame_{count}.jpg')
            cv2.imwrite(frame_filename, image)

            # Create a label file for the current frame
            label_filename = os.path.join(output_dir, f'{file[:-4]}_frame_{count}.txt')
            label_file = open(label_filename, 'w')

            # Write label information to the label file based on CSV data
            frame_data = df[df['Frame'] == count]
            for _, row in frame_data.iterrows():
                visibility = row['Ball']
                x = int(row['x'] * image.shape[1])  # Convert to pixel value
                y = int(row['y'] * image.shape[0])  # Convert to pixel value
                if visibility == 1:  # Adjust this condition based on your requirements
                    # Calculate the normalized coordinates and write them to the label file
                    label = f'0 {x / image.shape[1]} {y / image.shape[0]} 0.1 0.1\n'
                    label_file.write(label)

            # Close the label file for the current frame
            label_file.close()

            success, image = vidcap.read()
            count += 1

        # Release the VideoCapture object
        vidcap.release()

KeyboardInterrupt: 